In [ ]:
def create_graph_count_edges(gdf:gpd.GeoDataFrame, source_node:str, end_node:str):

    # create the graph for a single segment
    g = nx.from_pandas_edgelist(df = temp_gdf, source = 'f_intr_id', target = 't_intr_id', edge_attr=True)

    # count the number of edges per node.
    n_edge_list = []
    for nn in g.nodes():
        node_view = g[nn]  
        edge_count = len(node_view)        
        temp_list = [nn, edge_count]
        n_edge_list.append(temp_list)

    # build a dataframe that counts the number of edges per node:
    n_edge_count_df = pd.DataFrame(data = n_edge_list, columns = ['node_id', 'n_edges'])
    #print(n_edge_count_df.shape)
    
    # but now, we need to know which node is on which snd_group    
    # let's trim the temp_gdf
    sn_snd_group_id_df = temp_gdf.loc[temp_gdf['f_intr_id'].isin(n_edge_count_df['node_id']), ['f_intr_id', 'snd_group']]
    sn_snd_group_id_df.columns = ['node_id', 'snd_group']
    en_snd_group_id_df = temp_gdf.loc[temp_gdf['t_intr_id'].isin(n_edge_count_df['node_id']), ['t_intr_id', 'snd_group']]
    en_snd_group_id_df.columns = ['node_id', 'snd_group']

    node_snd_group_id_df = pd.concat(objs = [sn_snd_group_id_df, en_snd_group_id_df])
            
    # let's join the n_edge_count_df and the node_snd_group_id_df to get the snd on each node
    n_edge_count_df = pd.merge(left = n_edge_count_df, right = node_snd_group_id_df)        
    n_edge_count_df_agg = n_edge_count_df[['snd_group', 'n_edges']].groupby(['snd_group', 'n_edges']).agg(
        n_edges_group_count = ('n_edges', 'size')).reset_index()
    n_edge_count_df_agg = n_edge_count_df_agg.loc[n_edge_count_df_agg['n_edges'] == 1, :].copy()

    #n_edge_count_agg_df
    
    # now, let's just compute the max distance points
    edge_count_list = []
    for snd_group in n_edge_count_df['snd_group'].unique().tolist():
        
        temp_node_list = n_edge_count_df.loc[n_edge_count_df['snd_group'] == snd_group, 'node_id'].unique().tolist()            

        node_dist_list = []
        for sn_id, en_id in combinations(temp_node_list, 2):                        
            node_dist = points2distance(node_dict[sn_id], node_dict[en_id], unit = 'miles')
            node_dist_list.append([sn_id, en_id, node_dist])

        node_dist_df = pd.DataFrame(data = node_dist_list, 
                                    columns = ['sn_id', 'en_id', 'dist_miles'])
    
        # determine which are the farthest two nodes on an edge
        sn_id_agg_df = node_dist_df[['sn_id', 'dist_miles']].groupby(['sn_id']).agg(tot_dist = ('dist_miles', 'sum'),
                                                                                   node_count = ('dist_miles', 'size')).reset_index(names = 'node_id')
        
        en_id_agg_df = node_dist_df[['en_id', 'dist_miles']].groupby(['en_id']).agg(tot_dist = ('dist_miles', 'sum'),
                                                                                   node_count = ('dist_miles', 'size')).reset_index(names = 'node_id')        
        
        # combine and aggregate
        tot_node_dist_df = pd.concat(objs = [sn_id_agg_df, en_id_agg_df])
        tot_node_dist_df = tot_node_dist_df.groupby(['node_id']).agg(tot_dist = ('tot_dist', 'sum'),
                                                                    node_count = ('node_count', 'sum')).reset_index()
                        
        # rank distance
        tot_node_dist_df['tot_dist_rank'] = tot_node_dist_df['tot_dist'].rank(method = 'dense', ascending = False)
        tot_node_dist_df = tot_node_dist_df.sort_values(by = 'tot_dist', ascending = False)
        
        #print(tot_node_dist_df.head())
        # the top two are the ones to keep - get rid of 3+
        #tot_node_dist_df = tot_node_dist_df.loc[tot_node_dist_df['tot_dist_rank'] <= 2, :].copy()
        tot_node_dist_df['snd_group'] = snd_group
        edge_count_list.append(tot_node_dist_df)

    n_edge_dist_df = pd.concat(edge_count_list)

    print(n_edge_dist_df.head())
    print(n_edge_count_df.head())
    n_edge_count_df = pd.merge(left = n_edge_count_df, right = n_edge_dist_df)
    
    
    #print(edge_count_df.head())
    #print(edge_count_df['snd_group'].value_counts())
    # update the dataframes of interest    
    #n_edge_count_df = n_edge_count_df.loc[-n_edge_count_df['node_id'].isin(drop_node_list), :].copy()

    # put this into a dictionary - this will be used later
    node_snd_group_dict = {}
    for my_row in node_snd_group_id_df.itertuples():
        node_snd_group_dict[my_row.node_id] = my_row.snd_group 
    
    snd_group_node_dict = {}
    snd_group_id_list = n_edge_count_df['snd_group'].unique().tolist()
    for snd_group in snd_group_id_list:        
        snd_group_node_dict[snd_group] = n_edge_count_df.loc[n_edge_count_df['snd_group'] == snd_group, 'node_id'].tolist()
        

    return g, n_edge_count_df, node_snd_group_id_df, node_snd_group_dict, snd_group_node_dict